<a href="https://colab.research.google.com/github/yufujiaojiao/Deep-Learning-with-TensorFlow-book/blob/master/train_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import torch
import torchvision.datasets
from torch import nn
from torch.nn import Conv2d, MaxPool2d
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import time
print(torch.cuda.is_available())





#定义设备

device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

#准备数据集
train_data=torchvision.datasets.CIFAR10(root="./data",train=True,transform=torchvision.transforms.ToTensor(),download=True)


test_data=torchvision.datasets.CIFAR10(root="./",
                        train=False,
                        download=True,
                        transform=torchvision.transforms.ToTensor(),
                        )

#length 长度
train_data_size= len(train_data)
test_data_size = len(test_data)

print("训练数据集的长度为:{}".format(train_data_size))
print("测试集的长度为：{}".format(test_data_size))

#利用dataloader 加载数据集

train_dataloader= DataLoader(dataset=train_data,batch_size=64)
test_dataloader = DataLoader(dataset=test_data,batch_size=64)

#设计模型

class Tuidui(nn.Module):

    def __init__(self):
        super(Tuidui, self).__init__()

        self.model1=nn.Sequential(
            Conv2d(3,32,5,padding=2,stride=1),
            MaxPool2d(2),

            Conv2d(32,32,5,padding=2,stride=1),
            MaxPool2d(2),

            Conv2d(32,64,5,padding=2,stride=1),
            MaxPool2d(2),

            nn.Flatten(),
            nn.Linear(64*4*4,64),
            nn.Linear(64,10)

        )


    def forward(self,x):
        x=self.model1(x)
        return x


model=Tuidui()
# print(model)


model.to(device)

#定义损失函数

loss_fun=nn.CrossEntropyLoss()
# loss_fun.cuda()

loss_fun.to(device)

#定义优化器
learning_rate=0.01
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)

#定义一些训练参数
epoch=30


#添加tensorboard
writer = SummaryWriter("./logs_train")
for i in range(epoch):

    print("-----第{}轮训练开始了-----".format(i))

    total_train_step = 1
    total_test_step = 1

    model.train()#只对dropout，batchNormalization有用

    start_time = time.time()
    for data in train_dataloader:

        imgs,labels = data
        imgs=imgs.to(device)
        labels=labels.to(device)

        output=model(imgs)
        loss=loss_fun(output,labels)

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()


        if total_train_step%100==0:

            print("第{}步训练，loss为{}".format(total_train_step,loss.item()))
            writer.add_scalar("train_loss",loss.item(),total_train_step)
        total_train_step+=1
    end_time = time.time()
    print("1 epoch time:{}".format(end_time - start_time))
    total_test_loss=0
    total_test_accuracy=0
    model.eval()
    with torch.no_grad():
        for data in test_dataloader:
            imgs, labels =data

            imgs=imgs.to(device)
            labels=labels.to(device)

            output=model(imgs)
            test_loss=loss_fun(output,labels)
            total_test_loss=total_test_loss+test_loss

            preds=output.argmax(1)
            accuracy=(preds==labels).sum()
            total_test_accuracy=total_test_accuracy+accuracy

        print("epoch: {} test loss {}：".format(i, total_test_loss))
        print("total test accuracy {}".format(total_test_accuracy/test_data_size))

        writer.add_scalar("test_loss",total_test_loss,total_test_step)
        writer.add_scalar("test_accuracy",total_test_accuracy/test_data_size,total_test_step)
        total_test_step=total_test_step+1

    #保存模型
    torch.save(model.state_dict(),"tuidui_{}.pth".format(i))

writer.close()


True
Files already downloaded and verified
Files already downloaded and verified
训练数据集的长度为:50000
测试集的长度为：10000
-----第0轮训练开始了-----
第100步训练，loss为2.2968976497650146
第200步训练，loss为2.2846856117248535
第300步训练，loss为2.282973051071167
第400步训练，loss为2.225917339324951
第500步训练，loss为2.1581904888153076
第600步训练，loss为2.071760892868042
第700步训练，loss为1.9840805530548096
1 epoch time:11.064912557601929
epoch: 0 test loss 315.11285400390625：
total test accuracy 0.2838999927043915
-----第1轮训练开始了-----
第100步训练，loss为1.9422740936279297
第200步训练，loss为1.9317880868911743
第300步训练，loss为1.9001080989837646
第400步训练，loss为1.8362479209899902
第500步训练，loss为1.811966896057129
第600步训练，loss为1.8387666940689087
第700步训练，loss为1.789310097694397
1 epoch time:11.02589726448059
epoch: 1 test loss 284.9104309082031：
total test accuracy 0.34759998321533203
-----第2轮训练开始了-----
第100步训练，loss为1.7201042175292969
第200步训练，loss为1.7102936506271362
第300步训练，loss为1.6845002174377441
第400步训练，loss为1.6300231218338013
第500步训练，loss为1.6570967435836792
第600步训练，lo

# 新段落